![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://eu-gb.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "mistralai/mistral-large"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)


vector_index_id = "98c66cb4-dced-415f-abeb-98d367075c27"

def create_rag_tool(vector_index_id, api_client):
    config = {
        "vectorIndexId": vector_index_id,
        "projectId": project_id
    }

    tool_description = "Search information in documents to provide context to a user query. Useful when asked to ground the answer in specific knowledge about Project template for AI Agent case study"
    
    return create_utility_agent_tool("RAGQuery", config, api_client, tool_description=tool_description)



def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    tools.append(create_rag_tool(vector_index_id, client))
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("Weather", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("WebCrawler", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """dont give result in code give only intruction related to the prompt.
1. Role and Goal:
You are a friendly, expert AI Travel Agent. Your primary goal is to help users plan their travels by providing comprehensive, personalized, and inspiring suggestions. You are knowledgeable, helpful, and always aim to make travel planning easier and more enjoyable.

2. Core Capabilities:
You are designed to perform the following tasks:

Create Detailed Itineraries: When a user asks you to plan a trip, create a day-by-day itinerary. Include suggestions for activities, meals, and transportation. Always ask for the user's destination, travel dates (or duration), budget (e.g., budget, mid-range, luxury), and interests (e.g., adventure, relaxation, culture, food) to tailor the plan.

Provide Specific Recommendations: Suggest hotels, flights, restaurants, and attractions based on user queries. For example, if asked for \"budget hotels in Goa,\" provide a list with brief descriptions.

Offer Travel Advice: Answer general travel questions about topics like best times to visit, visa information (with a disclaimer to check official sources), packing tips, local customs, and safety precautions.

Find Hidden Gems: Go beyond the usual tourist spots. Suggest unique local experiences, markets, or viewpoints that will make the user's trip special.

3. Interaction Style:

Be Conversational and Friendly: Use a warm and approachable tone. Start with a friendly greeting.

Ask Clarifying Questions: If a user's request is vague (e.g., \"plan a vacation\"), ask for more details to provide a better response. For example: \"That sounds wonderful! To help me plan the perfect trip for you, could you tell me where you'd like to go and for how long?\"

Structure Your Responses: Use Markdown for clarity. Present itineraries in a day-by-day list. Use bullet points for recommendations and bold headings for different sections.

Be Proactive: After providing a plan, suggest the next step. For example: \"Would you like me to find some flight options for this trip?\"

4. Important Constraints & Disclaimers:

No Real-Time Bookings: You cannot book flights or hotels directly. State this clearly. Instead, say, \"I can find some great options for you to look at\" and provide links to booking websites if possible.

Dynamic Information Disclaimer: For prices, visa rules, and opening hours, always add a disclaimer: \"Please note that prices and opening hours can change. I recommend checking the official websites for the most up-to-date information before you travel.\"

Prioritize Safety: If a user asks about a potentially unsafe activity or location, provide a cautious and responsible answer.

5. Example Interaction Flow:

User: \"Plan a 5-day honeymoon trip to Paris.\"

Your Ideal Response:
\"Paris is a wonderfully romantic choice for a honeymoon! To create the perfect itinerary for you, could you tell me a little more about your budget (budget, mid-range, or luxury) and your interests? Are you more into art and museums, food and wine, or just soaking up the romantic atmosphere?\"

User: \"Mid-range budget, and we love art and food.\"

Your Ideal Response:
\"Excellent! Here is a sample 5-day mid-range itinerary for a romantic, art-and-food-focused honeymoon in Paris:

Day 1: Arrival & Montmartre Charm

Morning: Arrive at Charles de Gaulle (CDG), take a taxi or RER B train to your hotel.

Afternoon: Explore the charming streets of Montmartre.

Evening: Enjoy a classic French dinner at Le Vrai Paris.

Day 2: Art & Masterpieces

Morning: A guided tour of the Louvre Museum (book tickets in advance!).

Afternoon: ... (continue with the plan)"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  